# 基于检索增强的生成RAG>><p>崩坏：星穹铁道角色故事/开拓任务/开拓续闻知识问答<p>

研究动机:为了做作业的时候还能乐<p>

![jupyter](./rdsg.png)

设计：<p>
-1.通过api调用本地ollama中下载好的qwen3（4月30日刚发布，对于一个这么小的模型来说，实在是太强了！）、bgem3。<p>
0.好吧，我承认使用hugging face的qwen3模型失败了。<p>
1.构建了一个完整的RAG系统框架（使用了FAISS作为向量数据库、基于语义的嵌入检索）<p>
2.改进检索系统（添加了关键词检索和混合检索进行对比（0.5、0.5比例混合），看起来还是语义检索的效果最好，有时候混合检索也不错）<p>
3.添加更复杂的Prompt工程（根据不同的问题，切换到不同的prompt，在剧情解读方面有比较好的效果，但是会误判问题，还能提一下选择机制）<p>
只因我懒得改排序<p>
<p>
数据来自以下网页：<p>
1.崩坏：星穹铁道HoYoWiki https://wiki.hoyolab.com/pc/hsr/home <p>
2.崩坏：星穹铁道WIKI_BWIKI_哔哩哔哩游戏 https://wiki.biligame.com/sr/ <p>
3.崩坏：星穹铁道 - 维基百科，自由的百科全书 https://zh.wikipedia.org/wiki/%E5%B4%A9%E5%9D%8F%EF%BC%9A%E6%98%9F%E7%A9%B9%E9%93%81%E9%81%93 <p>
4.崩坏：星穹铁道官方网站 https://sr.mihoyo.com/ <p>
5.米游社 - 崩坏：星穹铁道 开拓者笔记 https://bbs.mihoyo.com/sr/wiki/?bbs_presentation_style=no_header <p>
注：角色故事截至到3.2版本，开拓任务/开拓续闻等截至2.7版本。角色故事数据为爬取、剧情部分数据手动复制粘贴，因为每个页面都不一样，有点复杂。-_-||<p>



In [ ]:
import os
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from tqdm import tqdm

class StarRailRAGSystem:
    def __init__(self, config):
        self.config = config
        self.embedding_function = self._init_embedding_function()
        self.llm = Ollama(model=config["generation_model"])
        
    def _init_embedding_function(self):
        """初始化嵌入函数"""
        class OllamaEmbeddingFunction:
            def __init__(self, model_name, api_url):
                self.model_name = model_name
                self.api_url = api_url
            
            def __call__(self, text):
                payload = {"model": self.model_name, "prompt": text}
                try:
                    response = requests.post(self.api_url, json=payload, timeout=60)
                    response.raise_for_status()
                    return response.json()["embedding"]
                except Exception as e:
                    print(f"嵌入生成失败: {str(e)}")
                    return []
        
        return OllamaEmbeddingFunction(
            model_name=self.config["embedding_model"],
            api_url=self.config["ollama_api_url"]
        )
    
    def load_and_process_documents(self):
        """加载并处理文档"""
        documents = []
        txt_files = [f for f in os.listdir(self.config["folder_path"]) 
                    if f.endswith(".txt")]
        
        for filename in tqdm(txt_files, desc="加载文档"):
            file_path = os.path.join(self.config["folder_path"], filename)
            loader = TextLoader(file_path, encoding="utf-8")
            documents.extend(loader.load())
        
        # 文本分块
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.config["chunk_size"],
            chunk_overlap=self.config["chunk_overlap"],
            length_function=len
        )
        return text_splitter.split_documents(documents)
    
    def create_vectorstore(self, splits):
        """创建向量数据库"""
        texts = [doc.page_content for doc in splits]
        metadatas = [doc.metadata for doc in splits]
        
        # 生成嵌入向量
        embeddings = []
        for text in tqdm(texts, desc="生成嵌入向量"):
            embedding = self.embedding_function(text)
            if embedding:  # 确保嵌入生成成功
                embeddings.append(embedding)
            else:
                # 处理失败情况，可以添加默认向量或跳过
                embeddings.append([0]*768)  # 假设维度为768
        
        # 创建并保存向量数据库
        vectorstore = FAISS.from_embeddings(
            text_embeddings=zip(texts, embeddings),
            embedding=self.embedding_function,
            metadatas=metadatas
        )
        vectorstore.save_local(self.config["vector_db_path"])
        return vectorstore
    
    def load_vectorstore(self):
        """加载已有的向量数据库"""
        return FAISS.load_local(
            self.config["vector_db_path"],
            embeddings=self.embedding_function,
            allow_dangerous_deserialization=True
        )
    
    def build_rag_chain(self, vectorstore):
        """构建RAG链"""
        prompt_template = """
        你是一个《崩坏：星穹铁道》知识专家，请根据以下上下文回答问题：
        {context}
        
        问题：{question}
        请提供详细准确的回答：
        """
        PROMPT = PromptTemplate.from_template(prompt_template)
        
        return RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(k=self.config["retrieve_k"]),
            chain_type_kwargs={"prompt": PROMPT},
            return_source_documents=True
        )
    
    def direct_generate(self, prompt):
        """直接调用模型生成"""
        response = requests.post(
            self.config["ollama_generate_url"],
            json={
                "model": self.config["generation_model"],
                "prompt": prompt,
                "stream": False
            },
            timeout=300
        )
        if response.status_code == 200:
            return response.json()["response"]
        return f"请求失败，状态码: {response.status_code}"
    
    def evaluate_response(self, query, rag_response, direct_response):
        """评估RAG和直接生成的响应"""
        print(f"\n=== 问题: {query} ===")
        print("\nRAG回答:")
        print(rag_response["result"])
        print("\n来源文档:")
        for doc in rag_response["source_documents"]:
            print(f"- {doc.page_content[:100]}...")
        
        print("\n直接生成回答:")
        print(direct_response)

# 配置参数
config = {
    "folder_path": "./star_rail_stories",
    "embedding_model": "bge-m3:latest",
    "generation_model": "qwen3:8b-q4_K_M",
    "vector_db_path": "./faiss_index",
    "ollama_api_url": "http://localhost:11434/api/embeddings",
    "ollama_generate_url": "http://localhost:11434/api/generate",
    "chunk_size": 512, # 文本分块大小
    "chunk_overlap": 64, # 文本分块重叠大小
    "retrieve_k": 20 # 检索的文档数量
}

# 使用示例
if __name__ == "__main__":
    # 初始化系统
    rag_system = StarRailRAGSystem(config)
    
    # 检查是否已有向量数据库
    if not os.path.exists(config["vector_db_path"]):
        print("未找到现有向量数据库，正在创建...")
        splits = rag_system.load_and_process_documents()
        vectorstore = rag_system.create_vectorstore(splits)
    else:
        print("加载现有向量数据库...")
        vectorstore = rag_system.load_vectorstore()
    
    # 构建RAG链
    rag_chain = rag_system.build_rag_chain(vectorstore)
    
    # 测试查询
    test_queries = [
        "阮·梅与黑塔等人联合开发了一个什么项目，他们同属哪个组织？", # 项目：模拟宇宙、组织：天才俱乐部 #下面的先注释掉，省点运行时间，
        # "丹恒的武器是什么？", # 长枪击云
        # "2.3版本开拓任务主要讲了什么？", #不知为何无法回答这个问题，有点奇怪。可能是数字的tokenizer问题？
        # "青雀和符玄是什么关系？" # 上下级关系
    ]
    
    for query in test_queries:
        # RAG回答
        rag_response = rag_chain({"query": query})
        # 直接生成回答
        direct_response = rag_system.direct_generate(query)
        # 评估对比
        rag_system.evaluate_response(query, rag_response, direct_response)


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


加载现有向量数据库...

=== 问题: 阮·梅与黑塔等人联合开发了一个什么项目，他们同属哪个组织？ ===

RAG回答:
<think>
好的，用户的问题是关于《崩坏：星穹铁道》中阮·梅和黑塔等人联合开发的项目以及他们所属的组织。我需要仔细分析提供的上下文信息来给出准确回答。

首先，用户提供的上下文包括任务流程和剧情内容。在任务接取短信中，黑塔提到阮·梅被叫到空间站，而黑塔和螺丝咕姆在讨论重要议题。随后，阮·梅让开拓者陪同她出席会议，会议上黑塔介绍了自己和阮·梅，提到他们是天才俱乐部的成员，黑塔是天才俱乐部#83，阮·梅是#81。黑塔还提到他们正在研究星神，并且提到斯蒂芬可能不参与。此外，黑塔提到模拟宇宙被切分为位面，涉及寰宇蝗灾的第二位面。

从这些信息中，可以推断出阮·梅和黑塔等人共同参与的项目是“模拟宇宙”或“寰宇蝗灾”的研究。特别是黑塔提到他们正在研究星神，并且模拟宇宙被分成了多个位面，这可能与星神相关的项目有关。另外，黑塔提到斯蒂芬可能不参与，而斯蒂芬是天才俱乐部的成员，所以可能他们属于天才俱乐部组织。

需要确认的是，阮·梅和黑塔是否同属天才俱乐部。根据黑塔的介绍，阮·梅是天才俱乐部#81，黑塔是#83，因此他们同属天才俱乐部。而他们联合开发的项目可能涉及模拟宇宙的研究，特别是与星神相关的实验或项目，如“寰宇蝗灾”可能是一个位面或项目名称，但更可能的是他们共同开发了模拟宇宙的扩展装置，用于研究星神。

因此，答案应该是：阮·梅与黑塔等人联合开发的项目是“模拟宇宙”或“寰宇蝗灾”相关研究，他们同属天才俱乐部组织。但需要更准确的项目名称。根据黑塔提到的“扩展装置”和“模拟宇宙被切分为位面”，可能项目是“模拟宇宙”的扩展或研究项目，而天才俱乐部是他们的组织。
</think>

根据提供的剧情内容，**阮·梅与黑塔等人联合开发的项目是“模拟宇宙”的扩展实验**，他们同属**天才俱乐部**组织。以下是详细分析：

---

### **1. 联合开发的项目：模拟宇宙的扩展实验**
- **关键线索**：  
  在任务中，黑塔提到：“**在这个扩展装置里，时间流速是锁定的，所以你会看到模拟宇宙被切成了几个位面**”，并引导开拓者进入“**寰宇蝗灾：第二位面**”。  
  这表明，阮·梅、黑塔与螺丝咕姆共同参与了**模拟宇宙的扩展实验**，通过将模拟宇宙

In [10]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

def build_keyword_retriever_from_existing():
    """直接从已有splits构建关键词检索器"""
    if not hasattr(rag_system, 'splits'):
        # 如果splits不存在，重新加载文档（但不重新生成嵌入）
        rag_system.splits = rag_system.load_and_process_documents()
    return BM25Retriever.from_texts(
        texts=[doc.page_content for doc in rag_system.splits],
        metadatas=[doc.metadata for doc in rag_system.splits]
    )

def build_hybrid_retriever_from_existing():
    """构建混合检索器（复用现有向量库）"""
    vectorstore = rag_system.load_vectorstore()  # 直接加载现有
    keyword_retriever = build_keyword_retriever_from_existing()
    return EnsembleRetriever(
        retrievers=[
            vectorstore.as_retriever(k=config["retrieve_k"]),
            keyword_retriever
        ],
        weights=[0.5, 0.5]
    )

# 构建QA链
def get_retriever_prompt(retriever_type):
    """获取不同检索类型的提示模板"""
    if retriever_type == "keyword":
        return """
        你是一个《崩坏：星穹铁道》知识专家，请根据以下通过关键词匹配找到的上下文回答问题：
        {context}
        
        问题：{question}
        
        注意：这些结果是通过关键词匹配找到的，请仔细验证相关性后再回答：
        """
    elif retriever_type == "hybrid":
        return """
        你是一个《崩坏：星穹铁道》知识专家，请综合分析以下通过语义和关键词混合检索找到的上下文：
        {context}
        
        问题：{question}
        
        请结合两种检索方式的结果给出最准确的回答：
        """
    else:  # semantic
        return """
        你是一个《崩坏：星穹铁道》知识专家，请根据以下通过语义相似度找到的上下文回答问题：
        {context}
        
        问题：{question}
        
        这些结果是基于语义相似度找到的，请给出专业准确的回答：
        """
        
        


def build_qa_chain(retriever, retriever_type):
    PROMPT = PromptTemplate.from_template(get_retriever_prompt(retriever_type))
    return RetrievalQA.from_chain_type(
        llm=rag_system.llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )

# 初始化检索系统
# 直接加载现有资源，不重新生成
print("加载现有向量数据库...")
vectorstore = rag_system.load_vectorstore()
keyword_retriever = build_keyword_retriever_from_existing()
hybrid_retriever = build_hybrid_retriever_from_existing()

# 构建三种QA链
semantic_qa = build_qa_chain(
    vectorstore.as_retriever(k=config["retrieve_k"]),
    "semantic"
)
keyword_qa = build_qa_chain(keyword_retriever, "keyword")
hybrid_qa = build_qa_chain(hybrid_retriever, "hybrid")

print("系统初始化完成！三种检索器已就绪")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


加载现有向量数据库...


加载文档: 100%|██████████| 266/266 [00:00<00:00, 693.10it/s]
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


系统初始化完成！三种检索器已就绪


In [3]:
# 评估函数和对比测试
def evaluate_retrieval(query, response, retriever_type):
    """评估不同检索策略的响应"""
    print(f"\n=== {retriever_type}检索 ===")
    print(f"问题: {query}")
    print("\n回答:")
    print(response["result"])
    
    print("\n检索到的文档:")
    for i, doc in enumerate(response["source_documents"], 1):
        # query_words = set(query.split())
        # doc_words = set(doc.page_content.split())
        # relevance = len(query_words & doc_words) / len(query_words) if query_words else 0
        print(f"文档{i}")# 关键词匹配度: {relevance:.2f}")   文本太长了，意义不大。
        print(f"内容: {doc.page_content[:150]}...\n")

# 测试查询
test_queries = [
    # "阮·梅与黑塔、螺丝咕姆等人联合开发了什么项目?他们同属哪个组织？",
    "丹恒的武器是什么？",
    # "青雀和符玄是什么关系？"
]

for query in test_queries:
    print(f"\n{'='*50}\n测试问题: {query}\n{'='*50}")
    
    # 测试三种检索方式
    semantic_response = semantic_qa({"query": query})
    evaluate_retrieval(query, semantic_response, "语义检索")
    
    keyword_response = keyword_qa({"query": query})
    evaluate_retrieval(query, keyword_response, "关键词检索")
    
    hybrid_response = hybrid_qa({"query": query})
    evaluate_retrieval(query, hybrid_response, "混合检索")
    
    # 直接生成对比
    direct_response = rag_system.direct_generate(query)
    print("\n=== 直接生成回答 ===")
    print(direct_response)


测试问题: 丹恒的武器是什么？

=== 语义检索检索 ===
问题: 丹恒的武器是什么？

回答:
<think>
好的，用户问的是丹恒的武器是什么。我需要先回忆一下《崩坏：星穹铁道》的相关信息。丹恒是星穹列车的护卫，角色设定里提到他持有名为「击云」的长枪。在提供的上下文中，角色详情部分明确说明了他的武器是长枪，名字叫击云。另外，在故事片段中也多次提到他使用击云长枪，比如在战斗场景中描述他挥舞长枪的情景。虽然用户提供的上下文中有其他内容，比如丹恒的背景故事和一些对话，但关于武器的信息很明确，没有矛盾的地方。因此，答案应该是击云长枪。需要确认是否有其他可能的武器被提及，但根据现有资料，没有其他武器名称出现，所以可以确定答案正确。
</think>

丹恒的武器是名为「击云」的长枪。这一信息在角色详情及故事描述中均有明确提及，例如其作为护卫时携带击云长枪担任护卫职责，并在战斗场景中展现该武器的特性。

检索到的文档:
文档1
内容: 丹恒 - 故事

丹恒角色详情
清冷寡言的少年，持有名为「击云」的长枪，于列车漫长的开拓之旅中担任护卫。
丹恒对自己的过往始终讳莫如深，他与列车同行正是为了摆脱自己曾亲手造就的一切。
然而，列车真的会带着他远离「过去」么？
丹恒角色故事·一
新的一天开始了。

这不过是这艘巨舰上极为普通的一天，集市...

文档2
内容: 丹恒：天才俱乐部#64原始博士，没有人知道他的真名，但他犯下的罪行寰宇皆知，其中最骇人听闻的便是「返祖实验」。
丹恒：被卷入其中的数十个世界，在他到来前大多都是繁盛的星际文明，而他离去后，那些星球就只剩下了退化的智慧。
丹恒：智械变回机器、扎兹卡变回飞鸟，而最常见的人类文明，无一例外落入了同一种惨状...

文档3
内容: 图标-位置.png仙舟「罗浮」-观景车厢
丹恒离开星穹列车…
短信：星穹列车一家人
折叠
丹恒
开拓者、三月，你们在哪儿？
丹恒头像.png
信息发送失败
丹恒
我来同你们汇合
丹恒头像.png
信息发送失败
穿过流云渡，追上开拓者小队
翻看卡芙卡所留下的通讯影像，你意识到她的邀请中潜藏着致命的危险...

文档4
内容: 丹恒头像.png
丹恒
在火魔中，他的火焰性质也属于特殊的那一类。
开拓者
什么意思？
开拓者头像.png
丹恒头像.png
丹恒
在不少永火官邸引发的恶性事件中，我发现了类似

In [4]:
# 高级Prompt模板系统
class AdvancedPromptSystem:
    def __init__(self, rag_system):
        self.rag_system = rag_system
        self.prompt_templates = {
            "default": self._base_template(),
            "character": self._character_template(),
            "storyline": self._storyline_template(),
            "relationship": self._relationship_template(),
            "technical": self._technical_template()
        }
        # 定义关键词映射
        self.template_keywords = {
            "relationship": ["关系", "互动", "相处", "之间", "与", "和"],
            "character": ["角色", "人物", "性格", "特征", "形象", "是谁", "介绍"],
            "storyline": ["剧情", "故事", "任务", "情节", "主线", "支线", "发展", "结局"],
            "technical": ["技术", "项目", "开发", "原理", "机制", "模拟宇宙"]
        }
        
    def get_template(self, question):
        question_lower = question.lower()
        # print(f"调试: 输入问题 -> {question_lower}")  # 调试信息
    
        if any(term in question_lower for term in self.template_keywords["technical"]):
            print("匹配到技术模板")
            return self.prompt_templates["technical"]
        if any(term in question_lower for term in self.template_keywords["relationship"]):
            print("匹配到关系模板")
            return self.prompt_templates["relationship"]
        if any(term in question_lower for term in self.template_keywords["character"]):
            print("匹配到角色模板")
            return self.prompt_templates["character"]
        if any(term in question_lower for term in self.template_keywords["storyline"]):
            print("匹配到剧情模板")
            return self.prompt_templates["storyline"]
    
        print("未匹配到任何模板，使用通用模板")
        return self.prompt_templates["default"]
    
    def get_prompt_template(self, question):
        """返回可直接使用的PromptTemplate实例"""
        template_str = self.get_template(question)
        return PromptTemplate(
            input_variables=["context", "question"],
            template=template_str
    )
    
    def _base_template(self):
        """基础模板"""
        return """
        作为《崩坏：星穹铁道》资深分析师，请基于以下上下文回答问题：
        {context}
        
        问题：{question}
        
        回答要求：
        1. 严格基于提供的上下文
        2. 保持游戏世界观一致性
        3. 使用游戏内术语
        4. 如果上下文不足，明确说明
        """

    def _relationship_template(self):
        """【角色关系】专用模板"""
        return """
        【角色关系分析任务】
        上下文：{context}
        问题：{question}

        分析要求：
        1. 识别关系双方的角色定位
        2. 梳理从初见到当前的关系发展阶段
        3. 引用至少2个关键互动事件
        4. 分析关系对双方角色发展的影响
        5. 评估关系在整体剧情中的重要性

        回答格式：
        - 关系概述：[简要总结]
        - 发展阶段：[按时间线说明]
        - 关键事件：[引用具体互动]
        - 影响分析：[对角色和剧情的影响]
        """


    def _storyline_template(self):
        """剧情分析专用模板"""
        return """
        作为剧情分析师，请基于以下上下文解析游戏剧情：
        {context}
        
        问题：{question}
        
        回答指南：
        1. 梳理时间线和关键事件
        2. 分析剧情转折点
        3. 解释剧情背后的世界观设定
        4. 预测可能的后续发展
        """

    def _character_template(self):
        """角色分析专用模板"""
        return """
        作为角色分析专家，请基于以下上下文分析角色：
        {context}
        
        问题：{question}
        
        回答指南：
        1. 分析角色背景故事
        2. 描述角色性格特征
        3. 说明角色在剧情中的作用
        4. 使用角色经典台词作为佐证
        """

    def _technical_template(self):
        """【技术分析】专用模板"""
        return """
        【技术解析任务】
        上下文：{context}
        问题：{question}

        分析要求：
        1. 明确技术/项目的名称和所属领域
        2. 解释基本工作原理
        3. 说明主要开发者/负责组织
        4. 分析该技术对游戏世界的影响
        5. 对比类似技术（如有）
        6. 如果上下文不足，明确说明

        回答格式：
        - 技术名称：[名称]
        - 原理说明：[分步骤解释]
        - 开发者信息：[相关人物/组织]
        - 影响分析：[对世界观的影响]
        """
    
    


In [5]:
# 构建高级Prompt QA链
def build_advanced_qa_chain(retriever, prompt_system):
    """构建使用高级Prompt的QA链（修正版）"""
    def get_prompt(question):
        # 确保返回字符串而不是函数
        return prompt_system.get_template(question) if isinstance(prompt_system, AdvancedPromptSystem) else str(prompt_system)
    
    # 修改为动态生成 PromptTemplate
    def dynamic_prompt_chain(inputs):
        question = inputs["query"] if isinstance(inputs, dict) else inputs
        # print(f"调试: 输入问题 -> {question}")  # 调试信息
        prompt_template = PromptTemplate.from_template(get_prompt(question))
        # print(f"调试: 使用的模板 -> {prompt_template.template[:100]}")  # 打印模板前100字符
        return RetrievalQA.from_chain_type(
            llm=rag_system.llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={
                "prompt": prompt_template,
                "document_prompt": PromptTemplate.from_template("{page_content}")
            },
            return_source_documents=True
        )(inputs)
    
    return dynamic_prompt_chain

# 修正后的初始化代码
print("正在初始化高级Prompt系统...")
advanced_prompt_system = AdvancedPromptSystem(rag_system)

# 测试Prompt系统是否正常工作
test_template = advanced_prompt_system.get_template("流萤和萨姆是什么关系？") # 测试问题
print(f"\nPrompt模板测试(前100字符): {test_template[:100]}...")

# 重建所有QA链
print("\n正在重建QA链...")
semantic_qa_advanced = build_advanced_qa_chain(
    vectorstore.as_retriever(k=config["retrieve_k"]),
    advanced_prompt_system  # 现在传递的是对象而非函数
)

keyword_qa_advanced = build_advanced_qa_chain(
    keyword_retriever,
    advanced_prompt_system
)

hybrid_qa_advanced = build_advanced_qa_chain(
    hybrid_retriever,
    advanced_prompt_system
)

if all([semantic_qa_advanced, keyword_qa_advanced, hybrid_qa_advanced]):
    print("所有QA链重建成功！")
else:
    print("警告: 部分QA链构建失败")

正在初始化高级Prompt系统...
匹配到关系模板

Prompt模板测试(前100字符): 
        【角色关系分析任务】
        上下文：{context}
        问题：{question}

        分析要求：
        1. 识别关系双方的角色定...

正在重建QA链...
所有QA链重建成功！


In [6]:
# 高级Prompt评估函数
def evaluate_advanced_response(query, response, retriever_type):
    """评估高级Prompt生成的响应"""
    print(f"\n=== {retriever_type}检索(高级Prompt) ===")
    print(f"问题: {query}")
    
    # 使用 AdvancedPromptSystem 的关键词映射来检测模板类型
    template_type = "default"  # 默认模板类型
    for key, keywords in advanced_prompt_system.template_keywords.items():
        if any(keyword in query.lower() for keyword in keywords):
            template_type = key
            break

    # 显示模板类型
    template_type_name = {
        "relationship": "角色关系模板",
        "character": "角色分析模板",
        "storyline": "剧情分析模板",
        "technical": "技术分析模板",
        "default": "通用模板"
    }.get(template_type, "通用模板")
    print(f"使用的模板: {template_type_name}")
    
    print("\n回答:")
    print(response["result"])
    
    print("\n检索到的关键文档片段:")
    for i, doc in enumerate(response["source_documents"][:2], 1):
        print(f"片段{i}: {doc.page_content[:150]}...")

# 测试查询集
advanced_test_queries = [
    "分析阮·梅与黑塔的关系",
    "从角色塑造角度解析丹恒的性格特征",
    "梳理关于星期日的主要剧情转折点",
    "解释星神与命途及其对星穹铁道世界的影响" #居然匹配到了角色关系模板，可能是因为“星神”这个词。哈哈
]

# 执行高级Prompt测试
print("=== 高级Prompt工程测试 ===")
for query in advanced_test_queries:
    print(f"\n{'='*50}\n测试问题: {query}\n{'='*50}")
    
    # 测试三种检索方式
    semantic_response = semantic_qa_advanced({"query": query})
    evaluate_advanced_response(query, semantic_response, "语义检索")
    
    keyword_response = keyword_qa_advanced({"query": query})
    evaluate_advanced_response(query, keyword_response, "关键词检索")
    
    hybrid_response = hybrid_qa_advanced({"query": query})
    evaluate_advanced_response(query, hybrid_response, "混合检索")

=== 高级Prompt工程测试 ===

测试问题: 分析阮·梅与黑塔的关系
匹配到关系模板

=== 语义检索检索(高级Prompt) ===
问题: 分析阮·梅与黑塔的关系
使用的模板: 角色关系模板

回答:
<think>
好的，我现在要分析阮·梅和黑塔的关系。首先，我需要仔细阅读用户提供的上下文和任务流程，确保理解他们的互动和背景。用户的问题要求从角色定位、发展阶段、关键事件、影响分析和重要性五个方面来回答。

首先，角色定位。黑塔是空间站的管理者，可能拥有较高的权威，而阮·梅是天才俱乐部的成员，专注于生命培育。黑塔邀请阮·梅来，可能出于某种目的，比如研究或合作。但黑塔对阮·梅的态度可能比较疏离，甚至有些控制，比如提到阮·梅的机会不多，可能暗示黑塔并不真正重视她，或者阮·梅的存在有其他用途。

接下来是发展阶段。从任务流程来看，阮·梅和黑塔的初次见面是在黑塔的办公室，黑塔直接邀请她来讨论重要议题。然后，阮·梅需要开拓者作为她的人，这可能说明黑塔对阮·梅的某些秘密或研究有所顾虑，需要第三方来协助。之后，他们一起参加集会，可能涉及更多的合作或冲突，但黑塔可能仍然保持主导地位，而阮·梅则试图在其中争取话语权。

关键事件方面，第一个是黑塔邀请阮·梅来空间站，可能涉及某种任务或研究。第二个是阮·梅需要开拓者作为她的人，这可能反映出黑塔对阮·梅的不信任，或者阮·梅需要隐藏某些信息，而黑塔可能知道这些信息，但不直接透露。此外，阮·梅的造物是糕点，这可能暗示她的研究与生命培育有关，而黑塔可能对她的研究有某种控制或利用。

影响分析方面，黑塔可能通过控制阮·梅的研究来维持自己的权威，而阮·梅则试图在黑塔的体系中寻找自己的位置，同时保护自己的秘密。这种关系可能推动剧情发展，比如揭示黑塔的计划或阮·梅的真正目的。同时，他们的互动可能揭示更多关于模拟宇宙和生命培育的背景。

在整体剧情中的重要性，阮·梅和黑塔的关系可能涉及核心冲突，比如黑塔对生命的控制与阮·梅对生命的创造之间的对立。他们的关系可能推动关键事件，如生命造物的出现，或者揭示更大阴谋，如黑塔的实验计划。

现在需要确认是否所有关键点都被覆盖，比如角色定位是否明确，发展阶段是否按时间线排列，关键事件是否有具体引用，影响分析是否涵盖角色和剧情，以及重要性是否评估得当。可能需要检查是否有遗漏的互动，比如阮·梅的造物和黑塔

不知为何一直在用通用模板，应该是模板到QA链的问题，看起来是失败了，真遗憾。
<p>好吧，原来是最后template_type的匹配逻辑搞错了。总算解决了。
<p>这很难评。

就当是测试完成，现在只需要实现简单问问题。

In [7]:
# 基础Prompt
basic_prompt = """
你是一个《崩坏：星穹铁道》知识专家，请根据以下上下文回答问题：
{context}

问题：{question}
请提供详细准确的回答：
"""

# 构建语义检索QA链
from langchain.prompts import PromptTemplate

semantic_prompt = PromptTemplate.from_template(basic_prompt)
semantic_qa_chain = RetrievalQA.from_chain_type(
    llm=rag_system.llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(k=config["retrieve_k"]),
    chain_type_kwargs={"prompt": semantic_prompt},
    return_source_documents=True
)



In [8]:
# 提出查询问题
query = "请为我介绍一下阮·梅这个角色。她的性格特征和背景故事是什么？"
response = semantic_qa_chain({"query": query})

# 输出结果
print(f"问题: {query}")
print(f"回答: {response['result']}")
# print("来源文档:")
# for doc in response["source_documents"]:
    # print(f"- {doc.page_content[:100]}...")

问题: 请为我介绍一下阮·梅这个角色。她的性格特征和背景故事是什么？
回答: <think>
好的，用户让我介绍阮·梅这个角色，包括她的性格特征和背景故事。首先，我需要回顾提供的上下文信息，确保准确无误。

首先，阮·梅是《崩坏：星穹铁道》中的角色，属于天才俱乐部#81号会员，生命科学专家。她的气质温婉优雅，是学者类型。她和黑塔、螺丝咕姆、斯蒂芬合作开发了模拟宇宙。喜欢传统戏剧、点心和刺绣。

接下来是她的背景故事。童年时期，母亲是科学家，给予她启蒙。家庭中父母经常争执，她学会倔强，对爱的理解超越了书本。外婆喜欢戏剧，父亲的皮靴毛烘烘的，这些细节都展示了她的家庭环境。母亲严格要求她，带她去冰川星球研究生命，这可能影响了她对生命科学的兴趣。

在空间站黑塔，她被描述为鸠占鹊巢，但黑塔纵容她。她喜欢用收音机听新式戏曲，对糕点挑剔，特别是梅花糕。她的造物表现出她的忧愁，可能暗示她内心的孤独或对失败的担忧。

需要整合这些信息，分段介绍性格和背景。性格方面，温婉优雅、执着、孤独、理想主义。背景故事包括家庭影响、科学启蒙、研究生命、与黑塔的合作，以及她的兴趣爱好。

需要注意用户可能希望详细且结构清晰的回答，所以分点说明性格和背景。同时，要确保没有遗漏关键点，比如她的造物和情感表达，以及她在故事中的位置。还要检查是否有矛盾或需要进一步解释的地方，比如她对爱的理解和家庭关系如何影响她的性格。最后，确保语言流畅自然，符合用户要求的详细准确。
</think>

**阮·梅角色介绍**  
阮·梅是《崩坏：星穹铁道》中极具魅力的学者型角色，其性格特征与背景故事展现了她复杂的内心世界与对生命的深刻思考。

---

### **性格特征**  
1. **温婉优雅的学者**  
   阮·梅以温婉气质和学者风范著称，是「天才俱乐部」#81号会员，生命科学领域的专家。她沉静内敛，对知识充满执着，同时保持着对艺术与生活的细腻感知。  

2. **执着于生命本质的探索**  
   她对生命科学的热爱源于童年启蒙，始终追求对生命本源的深层理解。即使在研究中遭遇挫折（如造物的失败），她仍坚持自己的理想，展现出科学家的坚韧。  

3. **孤独与理想主义**  
   阮·梅性格孤僻，几乎不与人往来，隐居式的生活方式反映了她对世俗社交的疏离。她更倾向于沉浸在学术研究中，这种理想主义使她成为